In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Slpit data into train and test
from sklearn.linear_model import LogisticRegression # Model language 
from sklearn.metrics import accuracy_score # Checking the preformance of the model



In [3]:
CC_data =  pd.read_csv(r"C:\Users\Ahmad\Downloads\Compressed\CCFD\creditcard.csv") # Importing CCFD table to pandas dataframe


In [4]:
CC_data.head() # Summary view of the table

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [20]:
CC_data['Class'].value_counts() #view data labels


0    284315
1       492
Name: Class, dtype: int64

In [5]:
#Spiliting the fraudluent data labels form the Non-fraudualent 
fraud_label = CC_data[CC_data.Class == 1] 
Legal_label = CC_data[CC_data.Class == 0]

In [6]:
CC_data.groupby("Class").mean() # Calculate the mean of the data labels and grouping them by the class label

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [7]:
Legal_label.Amount.describe() # statistical Summary of the Legal data label

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [8]:
fraud_label.Amount.describe() # statistical Summary of the fraudlent data label

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

From this line of code CC_data['Class'].value_counts() it shows that their is an significantly more "0" Legal Transactions(284315) to 1"" fradulent transactions (492), Balance is needed in order to properly train the model so taking a random sample of 492 from the lega data label would help. 

In [7]:
Legal_sample = Legal_label.sample(n=492) # Taking a random sample of 492 from the lega data label

In [8]:
LF_data =pd.concat((Legal_sample, fraud_label), axis=0) # Joining the Legal_sample data frame with fraud_label data frame

In [17]:
LF_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
224428,143786.0,0.218401,0.531727,-0.064482,-0.258554,0.305309,-1.158944,0.942943,-0.243013,0.133664,...,0.376735,1.081001,-0.096755,-0.026239,-0.413163,-0.152033,0.109515,0.102894,36.26,0
57902,48131.0,-2.543046,-2.902032,1.770390,-1.217310,1.709779,-1.853984,-1.385152,0.422041,1.359186,...,0.568922,0.777709,0.491167,-0.002744,0.092049,-0.745734,0.058376,0.204041,124.00,0
4407,3767.0,1.388157,-0.429866,0.682244,-0.443657,-1.151989,-1.024911,-0.696741,-0.218880,0.670370,...,-0.251814,-0.676241,0.149474,0.263594,0.166918,-0.516900,-0.025569,0.015763,19.70,0
245771,152901.0,2.028926,0.366727,-2.594878,1.242234,1.219444,-0.393313,0.492943,-0.103706,0.096290,...,-0.045285,-0.034486,-0.077164,-0.066122,0.505538,-0.502483,-0.018057,-0.039864,12.96,0
183609,125876.0,1.829605,-0.044688,-2.355177,1.184103,0.933502,-0.894529,1.056550,-0.515706,-0.276472,...,0.239898,0.473176,-0.198829,0.502837,0.585547,-0.449624,-0.067551,-0.036522,150.00,0


In [9]:
LF_data.groupby('Class').mean() # Check if the sample is good

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96202.002033,-0.047276,0.111518,-0.014415,-0.099257,0.061144,0.060386,0.005529,-0.052907,0.039672,...,-0.020946,-0.007837,0.018793,0.006505,0.05671,0.029451,-0.025109,-0.039581,0.003426,77.620772
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


In [13]:
X = LF_data.drop(columns='Class', axis=1) #Droping the column Class from data frame X
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
143019   85069.0  1.302378 -0.606529 -0.681986 -1.904603  1.326623  3.436312   
170594  120258.0 -0.448343 -2.154946  0.270417 -2.593332  0.470787 -1.279064   
87820    61859.0  0.689134 -0.549456 -0.004522  1.605802 -0.475250 -0.508558   
227379  145053.0  2.014456 -0.928607 -0.433808 -0.541926 -0.882319 -0.014980   
220001  141976.0 -6.845925  6.212955 -5.648353 -1.922814 -2.159442 -1.945487   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [16]:
Y = LF_data['Class'] # Data frame Y is represented by Column Class from LF_data

In [17]:
x_training,x_testing,y_training,y_testing = train_test_split(X,Y, test_size=.2, stratify=Y, random_state=2) # Creating four variables X presented by labels and y represented by features which will be spilt into train and testing data at random

In [19]:
model = LogisticRegression()

In [21]:
model.fit(x_training,y_training) # training the model

LogisticRegression()

In [ ]:
x_training_prediction = model.predict(x_training)
training_data_accuracy = accuracy_score(x_training_prediction,y_training)